In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Nome: Sormane do Nascimento Guimarães
# Matrícula: 2086324039

## Carrega DataSet Treinamento e Teste

In [ ]:
#Importa arquivos de testes e treinamento

df = pd.read_csv("../input/airline-passenger-satisfaction/train.csv")
test = pd.read_csv("../input/airline-passenger-satisfaction/test.csv")

df.shape, test.shape

#### Junta os DataSets

In [ ]:
# Juntando os dataframes
df_all = df.append(test)

df_all.shape

# Análise Exploratória e Transformação dos Dados

In [ ]:
#Verifica variáveis
df_all.info()

In [ ]:
#Porcentagem valores nulos dataset de treino
totalNuloTreino = df_all['Arrival Delay in Minutes'].isna().sum()
totalNuloTreino = (totalNuloTreino / df_all.index.size) * 100
totalNuloTreino

In [ ]:
#Porcentagem valores nulos dataset de teste
#totalNuloTeste = test['Arrival Delay in Minutes'].isna().sum()
#totalNuloTeste = (totalNuloTeste / test.index.size) * 100
#totalNuloTeste

In [ ]:
# Exclui valores nulos

df_all.fillna(0,inplace=True)

Há valores nulos na variável "Arrival Delay in Minutes". Por se tratar de uma uma pequena parcela dos DataSets, sendo um pouco mais de 0.3%, optei por excluir essas linhas.

#### Verificarei se há desbalanceamento dos valores da variável target que possa inviesar o modelo.

In [ ]:
#Balanceamento variável target dataset treino

df_all['satisfaction'].value_counts(normalize=True)

In [ ]:
#Balanceamento variável target dataset teste

#test['satisfaction'].value_counts(normalize=True)

In [ ]:
# Verifica balanceamento da variável target treinamento
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(df_all['satisfaction'])
plt.xticks()

In [ ]:
# Verifica balanceamento da variável target treinamento
#import seaborn as sns
#import matplotlib.pyplot as plt

#sns.countplot(test['satisfaction'])
#plt.xticks()

Os DataSet não apreenta desbalanceamento da variável target, portanto não será necessário um tratamento na variável target.

In [ ]:
df_all.info()

#### Análise bivariada

In [ ]:
cols = [ 'Inflight wifi service','Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service','Cleanliness']

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(30, 20))
for variable, subplot in zip(cols, ax.flatten()):
    sns.countplot(df_all[variable],hue=df_all['satisfaction'], ax=subplot)
    for label in subplot.get_xticklabels():
        label.set_rotation(90)

In [ ]:
cat_col =['Gender','Customer Type','Type of Travel','Class']

fig, ax = plt.subplots(1,4, figsize=(25, 7))
for variable, subplot in zip(cat_col, ax.flatten()):
    sns.countplot(df_all[variable],hue=df_all['satisfaction'], ax=subplot)

Com a análise bivariada, podemos observar a relação de quantidade das variáveis explicativas com a variável target. Com essa análise podemos observar alguns fatores, entre eles a causa e efeito, como podemos observar na relação entre a variável class e satisfaction, vemos que o maior número de passageiros insatisfeitos estão nas classes Eco, que entende-se ter menos conforto que a classe Bussiness Class. É claro que o fator citado acima não valerá para todas as relações, como exemplo, o sexo do passageiro.

#### Verificando a correlação entre as variáveis quantitativas

Iremos verificar a correlação entre as variáveis explicativas

Irei criar uma nova variável com a dumarização da variavel satisfaction para verificar a relação

In [ ]:
def satisfac(string):
    if string == 'satisfied': return 1
    else: return 0

df_all['satisfaction_cont'] =df_all['satisfaction'].apply(satisfac)

In [ ]:
# Exclui variaveis não necessárias

df_all.drop('id',axis=1,inplace=True)
df_all.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
#Grafico correlação

plt.figure(figsize=(20,15))
sns.heatmap(df_all.corr(),annot=True,cmap='coolwarm')
plt.tight_layout

Podemos verificar que há uma grande relação entre as variáveis Arrival Delay in Minutes e Departure Delay in Minutes, demostrando alta multicolariedade entre as duas.

#### Gráfico de barras demostrando as relações

In [ ]:
#Gráfico de Barras

plt.figure(figsize=(11,6))
df_all.corr()['satisfaction_cont'].sort_values().drop('satisfaction_cont').plot(kind='bar')

In [ ]:
#Dropa coluna criada
df_all.drop('satisfaction_cont',axis=1,inplace=True)

Verificamos o que já foi visto no gráfico de temperatura acima, as maiores e menores relações.
Assim, podemos verificar que a maior correção está entre Online Boarding e satisfaction

### Convertendo as variáveis do tipo object para o tipo inteiro.

#### Iremos realizar esse processo utilizando a função para dumarizar os valores. Nesse passo, também iremos excluir as linhas que contém valores nulos.
#### E as colunas que não utilizaremos no nosso modelo

In [ ]:
def altera_cat(data):
    data['Customer Type'] = data['Customer Type'].map({'Loyal Customer': 1, 'disloyal Customer': 0})
    data['Type of Travel'] = data['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
    data['Class'] = data['Class'].map({'Business':2, 'Eco Plus':1, 'Eco':0})
    data['satisfaction'] = data['satisfaction'].map({'neutral or dissatisfied':0, 'satisfied':1})
    data = pd.get_dummies(data)
    #data.fillna(0,inplace=True)
    #data.drop('id',axis=1,inplace=True)
    #data.drop('Unnamed: 0', axis=1, inplace=True)
    return data
df_all = altera_cat(df_all)
#test = altera_cat(test)

In [ ]:
#Verifica dataset após transformar dummy
df.shape, test.shape
df.info(), test.info()

#### Verificaremos a presença de outliers

In [ ]:
# Verifica outliers Atraso partida

plt.figure(figsize=(11,6))
plt.boxplot(df_all['Departure Delay in Minutes'])
plt.title('Atraso da partida')
plt.ylabel('Minutos')
plt.show()

In [ ]:
# Verifica outliers Atraso chegada

plt.figure(figsize=(11,6))
plt.boxplot(df_all['Arrival Delay in Minutes'])
plt.title('Atraso Chegada')
plt.ylabel('Minutos')
plt.show()

In [ ]:
# Verifica outliers Atraso chegada

plt.figure(figsize=(11,6))
plt.boxplot(df_all['Flight Distance'])
plt.title('Distância do Voo')
plt.ylabel('Distância')
plt.show()

#### Análisando os boxplots, poderia considerar outliers, porém, nesse momento considerei dados reais com um range alto. 
#### Nos passos abaixo criarei novas variáveis standarlizadas(padronizando), assim, iremos testar o modelo com essas novas variáveis.

In [ ]:
#Histograma variavel atraso na partida
df_all['Departure Delay in Minutes'].plot(kind='hist',ec='black')

In [ ]:
#Histograma variavel atraso na chegada
df_all['Arrival Delay in Minutes'].plot(kind='hist',ec='black')

In [ ]:
#Histograma variavel distancia do voo
df_all['Flight Distance'].plot(kind='hist',ec='black')

#### Podemos observar que os histógramas representar o que já foi citado acima, que devido a conter um range muito variado, não apresenta os histógramas normalizados.

## Separa DataSet Treino e Validação

In [ ]:
# Separando dataframe

# Importando o train_test_split
from sklearn.model_selection import train_test_split

# Separando treino e teste
train, test = train_test_split(df_all, test_size=0.20, random_state=42)

# Separando treino e validação
train, valid = train_test_split(train, test_size=0.20, random_state=42)

train.shape, valid.shape, test.shape

# Separando treino e validação
#train, valid = train_test_split(df, test_size=0.20, random_state=42)

#train.shape, valid.shape

### Seleciona variáveis que não serão target

In [ ]:
# definindo colunas de entrada
feats = [c for c in df_all.columns if c not in ['satisfaction']]

feats

## Treina Modelo

In [ ]:
# treinar o modelo

# Importando o modelo
from sklearn.ensemble import RandomForestClassifier

# Instanciar o modelo
rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

In [ ]:
# treinar o modelo
rf.fit(train[feats], train['satisfaction'])

In [ ]:
# Prevendo os dados de validação
preds_val = rf.predict(valid[feats])

preds_val

In [ ]:
valid['satisfaction'].head(3)

In [ ]:
valid['satisfaction'].tail(3)

In [ ]:
# Avaliando o desempenho do modelo

# Importando a metrica
from sklearn.metrics import accuracy_score

In [ ]:
# Acurácia das previsões de validação
accuracy_score(valid['satisfaction'], preds_val)

In [ ]:
# Prevendo dados de teste
preds_test = rf.predict(test[feats])

preds_test

In [ ]:
test['satisfaction'].head(3)

In [ ]:
test['satisfaction'].tail(3)

In [ ]:
# Acurácia das previsões de testes
accuracy_score(test['satisfaction'], preds_test)

### Houve uma acurácia bastante alta, tanto no modelo de treino quanto o de teste

# Métricas

In [ ]:
# Importando a metrica
from sklearn.metrics import mean_squared_log_error

In [ ]:
# Aplicando a metrica validação
mean_squared_log_error((valid['satisfaction']), (preds_val))**(1/2)

In [ ]:
# Aplicando a metrica teste
mean_squared_log_error((test['satisfaction']), (preds_test))**(1/2)

### Retornou um R2 baixo, em ambos os modelos

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

Irei gerar o gráfico de barras de correlação novamente, agora com as variáveis dumarizadas, para comparar com o gráfico de variáveis importantes

In [ ]:
#Grafico correlação

plt.figure(figsize=(11,6))
df_all.corr()['satisfaction'].sort_values().drop('satisfaction').plot(kind='bar')

Podemos observar que há uma relação entre a importância das variáveis e a relação que as variáveis explicativas possui em relação ao variável target

### Matrix confusão

In [ ]:
# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
import scikitplot as skplt

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(valid['satisfaction'], preds_val)

In [ ]:
# Matriz de Confusão - Dados de teste
skplt.metrics.plot_confusion_matrix(test['satisfaction'], preds_test)

### Padronizando algumas variáveis para análise

In [ ]:
# Copiando o dataframe
df_all2 = df_all.copy()

In [ ]:
stard = ['Age','Departure Delay in Minutes','Arrival Delay in Minutes','Flight Distance'] 

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
df_all2[stard]=scaler.fit_transform(df_all2[stard])
df_all2.head()

In [ ]:
#train[stard]=scaler.fit_transform(train[stard])
#train.head()

In [ ]:
#test[stard]=scaler.fit_transform(test[stard])
#test.head()

### Testa Modelo após padronização

In [ ]:
# Separando treino e teste
train2, test2 = train_test_split(df_all2, test_size=0.20, random_state=42)

# Separando treino e validação
train2, valid2 = train_test_split(train2, test_size=0.20, random_state=42)

train2.shape, valid2.shape, test2.shape

In [ ]:
# definindo colunas de entrada
feats2 = [c for c in df_all2.columns if c not in ['satisfaction']]

feats2

In [ ]:
# Instanciar o modelo
rf2 = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

# treinar o modelo
rf2.fit(train2[feats2], train2['satisfaction'])

In [ ]:
# Prevendo os dados de validação
preds_val2 = rf2.predict(valid2[feats2])

preds_val2

In [ ]:
# Prevendo dados de teste
preds_test2 = rf2.predict(test2[feats2])

preds_test2

In [ ]:
# Acurácia das previsões de validação
accuracy_score(valid2['satisfaction'], preds_val2)

In [ ]:
# Acurácia das previsões de testes
accuracy_score(test2['satisfaction'], preds_test2)

In [ ]:
# Aplicando a metrica validação
mean_squared_log_error((valid2['satisfaction']), (preds_val2))**(1/2)

In [ ]:
# Aplicando a metrica teste
mean_squared_log_error((test2['satisfaction']), (preds_test2))**(1/2)

Podemos verificar que não houve uma minuscula mundança ao padronizar as variáveis que possuem um range alto.

Tendo em vista que identificamos duas variávies explicativas com correlação alta, as Departure Delay in Minutes e Arrival Delay in Minutes, vou excluílas do dataset e testar novamente o modelo.

In [ ]:
# Copia Dataset

df_all3 = df_all2.copy()

In [ ]:
#Excluí variáveis dos datasets

df_all3.drop('Departure Delay in Minutes',axis=1,inplace=True)
df_all3.drop('Arrival Delay in Minutes', axis=1, inplace=True)

In [ ]:
df_all3.info()

#### Modelo após exclusão variáveis

In [ ]:
# Separando treino e teste
train3, test3 = train_test_split(df_all3, test_size=0.20, random_state=42)

# Separando treino e validação
train3, valid3 = train_test_split(train3, test_size=0.20, random_state=42)

train3.shape, valid3.shape, test3.shape



In [ ]:
# definindo colunas de entrada
feats3 = [c for c in df_all3.columns if c not in ['satisfaction']]

feats3

In [ ]:
# Instanciar o modelo
rf3 = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

# treinar o modelo
rf3.fit(train3[feats3], train3['satisfaction'])

In [ ]:
# Prevendo os dados de validação
preds_val3 = rf3.predict(valid3[feats3])

preds_val3

In [ ]:
# Prevendo dados de teste
preds_test3 = rf3.predict(test3[feats3])

preds_test3

In [ ]:
# Acurácia das previsões de validação
accuracy_score(valid3['satisfaction'], preds_val3)

In [ ]:
# Acurácia das previsões de testes
accuracy_score(test3['satisfaction'], preds_test3)

In [ ]:
# Aplicando a metrica validação
mean_squared_log_error((valid3['satisfaction']), (preds_val3))**(1/2)

In [ ]:
# Aplicando a metrica teste
mean_squared_log_error((test3['satisfaction']), (preds_test3))**(1/2)

Percebemos que também não houve grandes mudanças no modelo.

Criarei uma feature enginer com as idades, uma nova variável com faixa etária.

In [ ]:
df_all['Age'].max()

In [ ]:
df_all['Age'].min()

In [ ]:
condicao =[(df_all.Age>=100),(df_all.Age>=95) & (df_all.Age<=99),(df_all.Age>=90) & (df_all.Age<=94),(df_all.Age>=85) & 
(df_all.Age<=89),(df_all.Age>=80) & (df_all.Age<=84),(df_all.Age>=75) & (df_all.Age<=79),(df_all.Age>=70) & (df_all.Age<=74),(df_all.Age>=65) & 
(df_all.Age<=69),(df_all.Age>=60) & (df_all.Age<=64),(df_all.Age>=55) & (df_all.Age<=59),(df_all.Age>=50) & (df_all.Age<=54),(df_all.Age>=45) & 
(df_all.Age<=49),(df_all.Age>=40) & (df_all.Age<=44),(df_all.Age>=35) & (df_all.Age<=39),(df_all.Age>=30) & (df_all.Age<=34),(df_all.Age>=25) & 
(df_all.Age<=29),(df_all.Age>=20) & (df_all.Age<=24),(df_all.Age>=15) & (df_all.Age<=19),(df_all.Age>=10) & (df_all.Age<=14),(df_all.Age>=5) & 
(df_all.Age<=9),(df_all.Age<=4)]
classes = ['Acima de 100 anos', '95 a 99 anos', '90 a 94 anos', '85 a 89 anos', '80 a 84 anos', '75 a 79 anos', 
'70 a 74 anos', '65 a 69 anos', '60 a 64 anos', '55 a 59 anos', '50 a 54 anos', '45 a 49 anos', '40 a 44 anos', 
'35 a 39 anos', '30 a 34 anos', '25 a 29 anos', '20 a 24 anos', '15 a 19 anos', '10 a 14 anos', '05 a 09 anos', 
'00 a 04 anos']

df_all['FaixaEtaria'] = np.select(condicao, classes)

In [ ]:
df_all.fillna(0,inplace=True)

In [ ]:
df_all4 = df_all.copy()

In [ ]:
df_all4.info()

In [ ]:
df_all4['FaixaEtaria'].unique()

In [ ]:
df_all4['FaixaEtariaDummy'] = df_all4['FaixaEtaria'].map({'05 a 09 anos': 0, '10 a 14 anos': 1, '15 a 19 anos': 2, '20 a 24 anos': 3, '25 a 29 anos': 4, '30 a 34 anos': 5,
                                                         '35 a 39 anos': 6,'40 a 44 anos': 7, '45 a 49 anos': 8, '50 a 54 anos': 9, '55 a 59 anos': 10, '60 a 64 anos': 11, 
                                                          '65 a 69 anos': 12, '70 a 74 anos': 13, '75 a 79 anos': 14, '80 a 84 anos': 15,'85 a 89 anos': 16})

In [ ]:
#Exclui variavel Age e FaixaEtaria

df_all4.drop('Age', axis=1, inplace=True)
df_all4.drop('FaixaEtaria', axis=1, inplace=True)

In [ ]:
df_all4.info()

In [ ]:
# Separando dataframe

# Importando o train_test_split
from sklearn.model_selection import train_test_split

# Separando treino e teste
train4, test4 = train_test_split(df_all4, test_size=0.20, random_state=42)

# Separando treino e validação
train4, valid4 = train_test_split(train4, test_size=0.20, random_state=42)

train4.shape, valid4.shape, test4.shape

In [ ]:
# definindo colunas de entrada
feats4 = [c for c in df_all4.columns if c not in ['satisfaction']]

feats4

In [ ]:
# Instanciar o modelo
rf4 = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

# treinar o modelo
rf4.fit(train4[feats4], train4['satisfaction'])

In [ ]:
# Prevendo os dados de validação
preds_val4 = rf4.predict(valid4[feats4])

preds_val4

In [ ]:
# Prevendo dados de teste
preds_test4 = rf4.predict(test4[feats4])

preds_test4

In [ ]:
# Acurácia das previsões de validação
accuracy_score(valid4['satisfaction'], preds_val4)

In [ ]:
# Acurácia das previsões de teste
accuracy_score(test4['satisfaction'], preds_test4)

In [ ]:
# Aplicando a metrica validação
mean_squared_log_error((valid4['satisfaction']), (preds_val4))**(1/2)

In [ ]:
# Aplicando a metrica validação
mean_squared_log_error((test4['satisfaction']), (preds_test4))**(1/2)

Podemos observar que também não houve muitas mudanças na criação de uma feature engine por faixar etária.

Concluir que nesse DataSet as mudanças realizadas que realizei nas viariáveis explicativas não alterarm significamente o modelo. 

Acreditava que ao padronizar as variáveis que possuiam um range muito alto poderia alterar o modelo, mas percebemos que quase não houve alteração. 

Quando criei uma feature engine, utilizando a variável Age e criando uma faixa etária, acreditava que isso iria piorar o nosso modelo, mas, também com a padronização das variáveis, quase não houve uma mudança. 

Algo que percebi nesse trabalho foi, que grande parte das variáveis que possuiam uma boa correlação em relação a variável target foram as que mais impactaram o modelo, não consigo afirmar que se as variáveis tiver boa correlação com a variável target, estas serão as mais importantes. Ainda falando sobre a correlação, percebi que as variáveis explicativas que tinham uma correlação muito grande entre si(multicolariedade) não foram classificadas como variáveis importantes para o modelo.

Outro fator importante foi a análise bivariada, uma análise que retorna como uma variável target e uma variável explicativa se comportam uma com a outra, essa análise, realizada através de gráficos, mostrou que aquelas que se mostraram com valores bem distintos, ou seja, ou estava muito satisfeito ou bem pouco satisfeito, foram indicadas no gráfico de importância como as mais importantes.

Concluo que a análise exploratória, o tratamento dos dados, a criação de features engineres e o mais importante, o entendimento dos resultados dessa fase é, na minha visão, o processo mais importante, pois, o modelo só reflete aquilo que foi feito nessa fase mencionada.